In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "college-information-llm"

In [2]:
llm = ChatOpenAI()

In [3]:
from knowledgebase import TXTKnowledgeBase

In [4]:
kb=TXTKnowledgeBase(txt_source_folder_path='lxbd')

In [5]:
#kb.initiate_documents()

In [6]:
vector=kb.return_retriever_from_persistant_vector_db()

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>
Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever(search_kwargs={'k':4})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response_stream=retrieval_chain.stream({"input": "美国博士申请成绩单认证应该怎么处理"})
output={}
curr_key = None
for chunk in response_stream:
    for key in chunk:
        if key not in output:
            output[key] = chunk[key]
        else:
            output[key] += chunk[key]
        if key != curr_key:
            print(f"\n\n{key}: {chunk[key]}", end="", flush=True)
        else:
            print(chunk[key], end="", flush=True)
        curr_key = key